In [1]:
from chain import approx_hmc
from utils import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Test 1: Coarse grid, check if the code gives the correct full distribution

In [2]:
chain1 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=0.2, Nx=5)

mu: 0.200
sigma: 1.000
a: 4.000
z: 2.000
dt: 0.20000
dx: 0.80000
Nx: 5
shape of P: (7, 7)
p+, p0, p-: 0.18250, 0.68500, 0.13250


In [3]:
T = 1
dist1 = chain1.ExitDist(T)
dist2 = [chain1.ExitProb_dp1(T, i*chain1.dx) for i in range(chain1.Nx+1)]
dist3 = [chain1.ExitProb_dp2(T, i*chain1.dx) for i in range(chain1.Nx+1)]
dist4 = [chain1.ExitProb_logdp1(T, i*chain1.dx) for i in range(chain1.Nx+1)]
dist5 = [chain1.ExitProb_logdp2(T, i*chain1.dx) for i in range(chain1.Nx+1)]
dist6 = [chain1.ExitProb_test(T, i*chain1.dx) for i in range(chain1.Nx+1)]
print(*dist1[:5])
print(*dist2[:5])
print(*dist3[:5])
print(*dist4[:5])
print(*dist5[:5])
print(*dist6[:5])

/gpfs/home/sliu167/WorkSpace/ddm/Backup/chain.py:113: RuntimeWarning: divide by zero encountered in log
  logP = np.log(self.AdjMat.todense())
/gpfs/home/sliu167/WorkSpace/ddm/Backup/chain.py:118: RuntimeWarning: divide by zero encountered in log
  logtable[:, [t_step]] = b.max() + np.log(self.AdjMat @ np.exp(b - b.max()))
/gpfs/home/sliu167/WorkSpace/ddm/Backup/chain.py:119: RuntimeWarning: divide by zero encountered in log
  logprob = logsumexp(np.log(self.init_dist) + logtable[:, 0])


0.024898093798437496 0.17631948681103515 0.31627776333925783 0.2638825492796875 0.11532800625683592
0.024898093798437496 0.17631948681103515 0.31627776333925783 0.2638825492796875 0.11532800625683591
0.024898093798437496 0.17631948681103515 0.31627776333925783 0.2638825492796875 0.11532800625683591
0.024898093798437493 0.17631948681103515 0.31627776333925783 0.2638825492796875 0.1153280062568359
0.024898093798437496 0.17631948681103515 0.3162777633392579 0.2638825492796876 0.11532800625683592
0.024898093798437493 0.17631948681103532 0.3162777633392574 0.2638825492796874 0.11532800625683595


#### Test 2: Dense grid, compare running time of exit probability

In [4]:
chain2 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=0.001, Nx=80)

mu: 0.200
sigma: 1.000
a: 4.000
z: 2.000
dt: 0.00100
dx: 0.05000
Nx: 80
shape of P: (82, 82)
p+, p0, p-: 0.20201, 0.59998, 0.19801


In [5]:
T = 4
%timeit prob2 = chain2.ExitProb_dp1(T, 4)
%timeit prob3 = chain2.ExitProb_dp2(T, 4)
%timeit prob4 = chain2.ExitProb_logdp1(T, 4)
%timeit prob5 = chain2.ExitProb_logdp2(T, 4)

40.8 ms ± 436 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
562 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
120 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
123 ms ± 4.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
prob2 = chain2.ExitProb_dp1(T, 4)
prob3 = chain2.ExitProb_dp2(T, 4)
prob4 = chain2.ExitProb_logdp1(T, 4)
prob5 = chain2.ExitProb_logdp2(T, 4)
prob2, prob3, prob4, prob5

(7.873250893361756e-05,
 7.873250893361756e-05,
 7.873250893362534e-05,
 7.87325089336158e-05)

#### Test 3: Compare results as the grid becomes denser, check if it converges to the continuous case

In [9]:
dt = 0.001
chain0 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=5, verbose=False)
chain1 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=10, verbose=False)
chain2 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=20, verbose=False)
chain3 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=40, verbose=False)
chain4 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=80, verbose=False)
prob0 = chain0.ExitProb_logdp2(T=5, s=0)
prob1 = chain1.ExitProb_logdp2(T=5, s=0)
prob2 = chain2.ExitProb_logdp2(T=5, s=0)
prob3 = chain3.ExitProb_logdp2(T=5, s=0)
prob4 = chain4.ExitProb_logdp2(T=5, s=0)
print(wfpt(T=5, mu=0.2, a=4, z=2/4, err=1e-10))
prob0/dt, prob1/dt, prob2/dt, prob3/dt, prob4/dt

0.02547652896815553


(0.02643224301257986,
 0.02539651859230776,
 0.025458311713189366,
 0.025473112226127977,
 0.02547677223099034)

In [10]:
dt = 0.0002
chain0 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=5, verbose=False)
chain1 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=10, verbose=False)
chain2 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=20, verbose=False)
chain3 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=40, verbose=False)
chain4 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=80, verbose=False)
chain5 = approx_hmc(mu=0.2, sigma=1, a=4, z=2/4, dt=dt, Nx=160, verbose=False)
prob0 = chain0.ExitProb_logdp2(T=5, s=0)
prob1 = chain1.ExitProb_logdp2(T=5, s=0)
prob2 = chain2.ExitProb_logdp2(T=5, s=0)
prob3 = chain3.ExitProb_logdp2(T=5, s=0)
prob4 = chain4.ExitProb_logdp2(T=5, s=0)
prob5 = chain5.ExitProb_logdp2(T=5, s=0)
print(wfpt(T=5, mu=0.2, a=4, z=2/4, err=1e-10))
prob0/dt, prob1/dt, prob2/dt, prob3/dt, prob4/dt, prob5/dt

0.02547652896815553


(0.026431750901944617,
 0.025395291002294176,
 0.02545712908536354,
 0.025471940789884538,
 0.02547560359179506,
 0.025476516789387695)

#### Test 4: Maximum Likelihood Parameter Search
True paramater: $\mu=0.2, z=1.5$.

In [11]:
data = np.load('data.npy')
data.shape

FileNotFoundError: [Errno 2] No such file or directory: 'data.npy'

In [12]:
probs1d = np.loadtxt('results/probs1d.txt')
mu_list = np.arange(-1, 1, 0.01)

idx = np.argmax(probs1d)
mu_list[idx]


FileNotFoundError: results/probs1d.txt not found.

In [ ]:
plt.plot(mu_list, probs1d)
plt.show()

In [ ]:
num_ticks = 10
# the index of the position of yticks
yticks = np.linspace(0, len(depth_list) - 1, num_ticks, dtype=np.int)
# the content of labels of these yticks
yticklabels = [depth_list[idx] for idx in yticks

In [ ]:
probs2d = np.loadtxt('results/probs2d.txt')
mu_list = np.linspace(-1, 1, 21)
z_list = np.linspace(0.5, 3.5, 21)
idx = np.unravel_index(np.argmax(probs2d), probs2d.shape)

mu_list[idx[0]], z_list[idx[1]]


In [ ]:
hm = sns.heatmap(probs2d, cmap='hot')

from matplotlib.patches import Rectangle
hm.add_patch(Rectangle((idx[1],idx[0]),1,1, fill=False, edgecolor='blue', lw=1))

hm.invert_yaxis()
hm.set_xlabel('$z$', fontsize=10)
hm.set_ylabel('$\mu$', fontsize=10)
hm.set_xticks(np.arange(0, 21, 5))
hm.set_yticks(np.arange(0, 21, 5))

hm.set_xticklabels(z_list[::5])
hm.set_yticklabels(mu_list[::5])
plt.show()